In [13]:
"""
EvGNN Model Testing Notebook
Load a trained model and evaluate on test dataset
"""

# ============================================================================
# CONFIGURATION
# ============================================================================

# Model Configuration
MODEL_NAME = 'evgnn_ncars_fuse.pth'  # Model file to load
MODEL_PATH = '../results/TrainedModels'  # Path to trained models
CONV_TYPE = "fuse"  # "fuse" or "ori_aegnn" - must match the trained model

# Dataset Configuration
DATASET = 'ncars'  # 'ncars' or 'ncaltech'
NCARS_PATH = r'/Users/hannes/Documents/University/Datasets/raw_ncars/Prophesee_Dataset_n_cars'
NCALTECH_PATH = r'/Users/hannes/Documents/University/Datasets/raw_ncaltech'

# Test Configuration
BATCH_SIZE = 32  # Batch size for testing
N_SAMPLES = 10000  # Number of events to sample per recording
BETA = 0.5e-5  # Time normalization
RADIUS = 3.0  # Graph radius
MAX_NEIGHBORS = 16  # Max neighbors in graph

# Device Configuration
USE_DEVICE = 'cpu'  # 'auto', 'mps', 'cuda', or 'cpu'
CPU_THREADS = 8

# Output Configuration
SAVE_RESULTS = True  # Whether to save results to JSON
RESULTS_DIR = '../results/sync_test_results'


In [14]:
# ============================================================================
# SETUP
# ============================================================================

import sys, os
import torch
import json

# Set MPS fallback BEFORE importing torch
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Add project to path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.Datasets.batching import BatchManager
from src.Datasets.ncaltech101 import NCaltech
from src.Datasets.ncars import NCars
from src.Models.CleanEvGNN.recognition import RecognitionModel as EvGNN
from torch_geometric.data import Data as PyGData

print("✓ Imports successful")


✓ Imports successful


In [15]:
# ============================================================================
# DEVICE SETUP
# ============================================================================

torch.set_num_threads(CPU_THREADS)

if USE_DEVICE == 'cpu':
    device = torch.device("cpu")
    print(f"Using device: cpu with {CPU_THREADS} threads (forced)")
elif USE_DEVICE == 'mps':
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print(f"Using device: mps (Apple Silicon GPU with CPU fallback for unsupported ops)")
        print(f"CPU operations will use {CPU_THREADS} threads")
    else:
        device = torch.device("cpu")
        print(f"MPS not available, falling back to cpu with {CPU_THREADS} threads")
elif USE_DEVICE == 'cuda':
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using device: cuda")
    else:
        device = torch.device("cpu")
        print(f"CUDA not available, falling back to cpu with {CPU_THREADS} threads")
else:  # 'auto'
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print(f"Using device: mps (Apple Silicon GPU with CPU fallback)")
        print(f"CPU operations will use {CPU_THREADS} threads")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using device: cuda")
    else:
        device = torch.device("cpu")
        print(f"Using device: cpu with {CPU_THREADS} threads")


Using device: cpu with 8 threads (forced)


In [16]:
# ============================================================================
# DATASET SETUP
# ============================================================================

# Get dataset info
if DATASET == 'ncars':
    num_classes = len(NCars.get_info().classes)
    image_size = NCars.get_info().image_size
    dataset_path = NCARS_PATH
elif DATASET == 'ncaltech':
    num_classes = len(NCaltech.get_info().classes)
    image_size = NCaltech.get_info().image_size
    dataset_path = NCALTECH_PATH
else:
    raise ValueError(f"Unknown dataset: {DATASET}")

print(f"\nDataset: {DATASET}")
print(f"Number of classes: {num_classes}")
print(f"Image size: {image_size}")



Dataset: ncars
Number of classes: 2
Image size: (100, 120)


In [17]:
# ============================================================================
# MODEL SETUP
# ============================================================================

print(f"\n{'='*70}")
print("LOADING MODEL")
print(f"{'='*70}")

# Create model architecture
img_shape_for_model = (image_size[1], image_size[0])  # Swap to (width, height)

model = EvGNN(
    network="graph_res",
    dataset=DATASET,
    num_classes=num_classes,
    img_shape=img_shape_for_model,
    dim=3,
    conv_type=CONV_TYPE,
    distill=False
).to(device)

# Define transform function
def transform_graph(graph: PyGData) -> PyGData:
    return model.data_transform(
        graph,
        n_samples=N_SAMPLES,
        sampling=True,
        beta=BETA,
        radius=RADIUS,
        max_neighbors=MAX_NEIGHBORS
    ).to(device)

# Load trained weights
model_full_path = os.path.join(MODEL_PATH, MODEL_NAME)
if not os.path.exists(model_full_path):
    raise FileNotFoundError(f"Model not found: {model_full_path}")

print(f"Loading model from: {model_full_path}")
model.load_state_dict(torch.load(model_full_path, map_location=device))
model.eval()
print("✓ Model loaded successfully")
print(f"Architecture: {CONV_TYPE}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")



LOADING MODEL
Loading model from: ../results/TrainedModels/evgnn_ncars_fuse.pth
✓ Model loaded successfully
Architecture: fuse
Parameters: 6,608


In [18]:
# ============================================================================
# LOAD TEST DATASET
# ============================================================================

print(f"\n{'='*70}")
print("LOADING TEST DATASET")
print(f"{'='*70}")

# Create dataset with transform
if DATASET == 'ncaltech':
    dataset_obj = NCaltech(
        root=dataset_path,
        transform=transform_graph
    )
elif DATASET == 'ncars':
    dataset_obj = NCars(
        root=dataset_path,
        transform=transform_graph
    )

# Process test set
dataset_obj.process(modes=["test"])
num_test_samples = dataset_obj.get_mode_length("test")
print(f"Test samples: {num_test_samples}")

# Create test data loader
test_loader = BatchManager(
    dataset=dataset_obj,
    batch_size=BATCH_SIZE,
    mode="test"
)

print("✓ Test dataset loaded")



LOADING TEST DATASET


test (cars):   0%|          | 0/4396 [00:00<?, ?it/s]

test (background):   0%|          | 0/4211 [00:00<?, ?it/s]

Test samples: 8607
✓ Test dataset loaded


In [19]:
# ============================================================================
# EVALUATE ON TEST SET
# ============================================================================

print(f"\n{'='*70}")
print("EVALUATING ON TEST SET")
print(f"{'='*70}")

model.eval()
correct = 0
total = 0

num_test_batches = (num_test_samples + BATCH_SIZE - 1) // BATCH_SIZE

print(f"Processing {num_test_batches} batches...")

with torch.no_grad():
    for batch_idx in range(num_test_batches):
        # Get batch
        batch = next(test_loader)
        targets = batch.y.to(device)

        # Forward pass
        outputs = model(batch)
        predictions = outputs.argmax(dim=-1)

        # Count correct predictions
        correct += (predictions == targets).sum().item()
        total += targets.size(0)

        # Print progress every 10 batches
        if (batch_idx + 1) % 10 == 0:
            current_acc = correct / total
            print(f"  Batch {batch_idx + 1}/{num_test_batches} - "
                  f"Accuracy so far: {current_acc:.4f} ({current_acc*100:.2f}%)")

# Calculate final accuracy
test_accuracy = correct / total

print(f"\n{'='*70}")
print(f"TEST RESULTS")
print(f"{'='*70}")
print(f"Total samples: {total}")
print(f"Correct predictions: {correct}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"{'='*70}")



EVALUATING ON TEST SET
Processing 269 batches...
  Batch 10/269 - Accuracy so far: 0.8469 (84.69%)
  Batch 20/269 - Accuracy so far: 0.8547 (85.47%)
  Batch 30/269 - Accuracy so far: 0.8594 (85.94%)
  Batch 40/269 - Accuracy so far: 0.8617 (86.17%)
  Batch 50/269 - Accuracy so far: 0.8656 (86.56%)
  Batch 60/269 - Accuracy so far: 0.8641 (86.41%)


KeyboardInterrupt: 

In [ ]:
# ============================================================================
# SAVE RESULTS
# ============================================================================

if SAVE_RESULTS:
    print(f"\nSaving results...")

    # Create results dictionary
    results = {
        'model_name': MODEL_NAME,
        'dataset': DATASET,
        'architecture': CONV_TYPE,
        'test_accuracy': float(test_accuracy),
        'num_test_samples': total,
        'correct_predictions': correct,
        'batch_size': BATCH_SIZE,
        'configuration': {
            'n_samples': N_SAMPLES,
            'beta': BETA,
            'radius': RADIUS,
            'max_neighbors': MAX_NEIGHBORS
        }
    }

    # Create results directory
    os.makedirs(RESULTS_DIR, exist_ok=True)

    # Save to JSON
    result_filename = f'test_results_{DATASET}_{CONV_TYPE}.json'
    result_path = os.path.join(RESULTS_DIR, result_filename)

    with open(result_path, 'w') as f:
        json.dump(results, f, indent=2)

    print(f"✓ Results saved to: {result_path}")
else:
    print("\nResults not saved (SAVE_RESULTS = False)")

print("\n✓ Testing complete!")
